# Description

This notebook analyzes the LVs driving the association of Niacin with some cardiovascular traits.

# Modules loading

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path

import numpy as np
import pandas as pd

from entity import Gene
import conf

# Settings

In [3]:
QUANTILE = 0.95

# Paths

In [4]:
OUTPUT_DIR = conf.RESULTS["DRUG_DISEASE_ANALYSES"] / "lincs" / "analyses"
display(OUTPUT_DIR)
OUTPUT_DIR.mkdir(exist_ok=True, parents=True)

PosixPath('/media/miltondp/Elements1/projects/phenoplier/results/drug_disease_analyses/lincs/analyses')

In [5]:
INPUT_DIR = conf.RESULTS["DRUG_DISEASE_ANALYSES"] / "lincs" / "predictions"
# display(OUTPUT_DIR)
# OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
input_predictions_by_tissue_file = INPUT_DIR / "full_predictions_by_tissue-rank.h5"
display(input_predictions_by_tissue_file)

PosixPath('/media/miltondp/Elements1/projects/phenoplier/results/drug_disease_analyses/lincs/predictions/full_predictions_by_tissue-rank.h5')

# Data loading

## PharmacotherapyDB: load gold standard

### Final

In [6]:
gold_standard = pd.read_pickle(
    Path(conf.RESULTS["DRUG_DISEASE_ANALYSES"], "gold_standard.pkl"),
)

In [7]:
gold_standard.shape

(998, 3)

In [8]:
gold_standard.head()

,trait,drug,true_class
0,DOID:10652,DB00843,1
1,DOID:10652,DB00674,1
2,DOID:10652,DB01043,1
3,DOID:10652,DB00989,1
4,DOID:10652,DB00810,0


### Info

In [9]:
# TODO hardcoded
input_file = Path(
    conf.DATA_DIR, "hetionet/pharmacotherapydb-v1.0", "indications.tsv"
).resolve()
display(input_file)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/bases_data/base_orig/data/hetionet/pharmacotherapydb-v1.0/indications.tsv')

In [10]:
gold_standard_info = pd.read_csv(input_file, sep="\t")

In [11]:
gold_standard_info = gold_standard_info.rename(columns={"drug": "drug_name"})

In [12]:
gold_standard_info.shape

(1388, 7)

In [13]:
gold_standard_info.head()

,doid_id,drugbank_id,disease,drug_name,category,n_curators,n_resources
0,DOID:10652,DB00843,Alzheimer's disease,Donepezil,DM,2,1
1,DOID:10652,DB00674,Alzheimer's disease,Galantamine,DM,1,4
2,DOID:10652,DB01043,Alzheimer's disease,Memantine,DM,1,3
3,DOID:10652,DB00989,Alzheimer's disease,Rivastigmine,DM,1,3
4,DOID:10652,DB00245,Alzheimer's disease,Benzatropine,SYM,3,1


In [14]:
gold_standard_info = (
    gold_standard.set_index(["trait", "drug"])
    .join(
        gold_standard_info.rename(
            columns={"doid_id": "trait", "drugbank_id": "drug"}
        ).set_index(["trait", "drug"])
    )
    .reset_index()
)

In [15]:
gold_standard_info.shape

(998, 8)

In [16]:
gold_standard_info.head()

,trait,drug,true_class,disease,drug_name,category,n_curators,n_resources
0,DOID:10652,DB00843,1,Alzheimer's disease,Donepezil,DM,2,1
1,DOID:10652,DB00674,1,Alzheimer's disease,Galantamine,DM,1,4
2,DOID:10652,DB01043,1,Alzheimer's disease,Memantine,DM,1,3
3,DOID:10652,DB00989,1,Alzheimer's disease,Rivastigmine,DM,1,3
4,DOID:10652,DB00810,0,Alzheimer's disease,Biperiden,NOT,2,1


## LINCS data

In [17]:
input_file = Path(
    conf.RESULTS["DRUG_DISEASE_ANALYSES"], "lincs", "lincs-data.pkl"
).resolve()

display(input_file)

PosixPath('/media/miltondp/Elements1/projects/phenoplier/results/drug_disease_analyses/lincs/lincs-data.pkl')

In [18]:
lincs_data = pd.read_pickle(input_file).T.rename(columns=Gene.GENE_ID_TO_NAME_MAP)

In [19]:
display(lincs_data.shape)

(1170, 7120)

In [20]:
display(lincs_data.head())

,ADA,CDH2,AKT3,MED6,ACOT8,ABI1,GNPDA1,CDH3,TANK,TOPORS-AS1,...,RBX1,CDC42,THOC1,RCE1,HNRNPDL,DMTF1,PPP4R1,CDH1,PTBP3,CASP8AP2
perturbagen,,,,,,,,,,,,,,,,,,,,,
DB00014,-1.001,1.146,-0.693,-0.037,0.162,-0.380,0.649,-1.952,0.294,0.274,...,-0.592,0.178,0.270,-0.013,0.351,-0.128,0.274,-0.734,-0.048,0.036
DB00091,-1.835,-1.863,1.694,0.383,-0.899,0.628,-4.878,2.527,1.709,-0.981,...,-0.668,-2.795,-0.333,-0.027,0.578,6.926,-1.875,4.382,0.266,-4.995
DB00121,1.391,0.011,-0.804,0.269,0.105,-0.588,-1.899,0.306,-1.178,0.120,...,-0.962,0.450,-0.999,1.358,-1.476,0.423,-1.356,-1.897,-0.299,-0.732
DB00130,1.132,-1.020,-0.164,-0.997,-0.090,0.195,-2.341,0.494,-0.813,-1.140,...,-0.553,-0.528,0.308,0.534,-0.320,3.001,-0.414,0.134,0.147,2.158
DB00131,0.257,1.143,1.145,0.185,-1.291,-0.457,0.038,1.442,-1.692,0.593,...,-0.556,-0.600,0.092,-0.564,0.174,-1.203,-1.080,-0.575,-0.792,-0.095


## LINCS projection

In [21]:
input_file = Path(
    conf.RESULTS["DRUG_DISEASE_ANALYSES"], "lincs", "lincs-projection.pkl"
).resolve()

display(input_file)

PosixPath('/media/miltondp/Elements1/projects/phenoplier/results/drug_disease_analyses/lincs/lincs-projection.pkl')

In [22]:
lincs_projection = pd.read_pickle(input_file).T

In [23]:
display(lincs_projection.shape)

(1170, 987)

In [24]:
display(lincs_projection.head())

,LV1,LV2,LV3,LV4,LV5,LV6,LV7,LV8,LV9,LV10,...,LV978,LV979,LV980,LV981,LV982,LV983,LV984,LV985,LV986,LV987
perturbagen,,,,,,,,,,,,,,,,,,,,,
DB00014,0.036115,0.012281,-0.005533,-0.004151,-0.015156,-0.010861,0.008181,-0.008148,0.004948,-0.009869,...,-0.003139,0.003546,-0.006242,0.002449,-0.004391,-0.004334,-0.019902,0.004292,-0.010378,0.002000
DB00091,-0.358610,0.009738,-0.059174,-0.032884,-0.005276,0.063369,-0.079522,-0.018980,-0.025782,0.021540,...,-0.019892,0.003610,-0.016165,0.008686,0.065335,-0.042051,0.065516,0.016261,0.056314,-0.051035
DB00121,0.091067,0.004650,0.013454,0.005934,0.025747,0.004324,-0.001865,-0.019613,0.051503,0.013032,...,0.001629,-0.012565,0.009613,-0.014404,-0.009188,0.006544,0.047466,-0.016220,-0.018844,0.034675
DB00130,0.008923,-0.006554,0.008906,-0.003991,0.000346,0.008212,0.001242,-0.012771,0.014221,0.008269,...,-0.026139,-0.002977,0.021969,0.001204,-0.012731,-0.000076,-0.029127,-0.022679,0.016179,-0.003029
DB00131,0.046469,0.021758,-0.004939,-0.028524,-0.013932,0.012231,-0.001325,0.003787,0.035189,-0.004377,...,0.017373,0.005304,-0.001649,0.004017,-0.000622,-0.006477,0.021663,0.000988,0.002038,-0.011747


## MultiPLIER Z

In [25]:
# multiplier_z = pd.read_pickle(conf.MULTIPLIER["MODEL_Z_MATRIX_FILE"])

In [26]:
# multiplier_z.shape

In [27]:
# multiplier_z.head()

# Niacin and cardiovascular diseases

In [28]:
from entity import Trait

In [29]:
Trait.get_traits_from_efo("atherosclerosis")

[I70-Diagnoses_main_ICD10_I70_Atherosclerosis]

In [30]:
Trait.get_traits_from_efo("coronary artery disease")

[I25-Diagnoses_main_ICD10_I25_Chronic_ischaemic_heart_disease,
 CARDIoGRAM_C4D_CAD_ADDITIVE]

In [31]:
_phenomexcan_traits = [
    "I70-Diagnoses_main_ICD10_I70_Atherosclerosis",
    "CARDIoGRAM_C4D_CAD_ADDITIVE",
    "I25-Diagnoses_main_ICD10_I25_Chronic_ischaemic_heart_disease",
    "20002_1473-Noncancer_illness_code_selfreported_high_cholesterol",
    "6150_100-Vascularheart_problems_diagnosed_by_doctor_None_of_the_above",
    "6150_1-Vascularheart_problems_diagnosed_by_doctor_Heart_attack",
    "I9_CHD-Major_coronary_heart_disease_event",
    "I9_CORATHER-Coronary_atherosclerosis",
    "I9_IHD-Ischaemic_heart_disease_wide_definition",
    "I9_MI-Myocardial_infarction",
    "I21-Diagnoses_main_ICD10_I21_Acute_myocardial_infarction",
    "20002_1075-Noncancer_illness_code_selfreported_heart_attackmyocardial_infarction",
]

_drug_id = "DB00627"
_drug_name = "Niacin"

In [32]:
for p in _phenomexcan_traits:
    print(p)
    d = Trait.get_trait(full_code=p)
    print((d.n, d.n_cases))

    print("\n")

I70-Diagnoses_main_ICD10_I70_Atherosclerosis
(361194, 566.0)


CARDIoGRAM_C4D_CAD_ADDITIVE
(184305, 60801.0)


I25-Diagnoses_main_ICD10_I25_Chronic_ischaemic_heart_disease
(361194, 12769.0)


20002_1473-Noncancer_illness_code_selfreported_high_cholesterol
(361141, 43957.0)


6150_100-Vascularheart_problems_diagnosed_by_doctor_None_of_the_above
(360420, 253565.0)


6150_1-Vascularheart_problems_diagnosed_by_doctor_Heart_attack
(360420, 8288.0)


I9_CHD-Major_coronary_heart_disease_event
(361194, 10157.0)


I9_CORATHER-Coronary_atherosclerosis
(361194, 14334.0)


I9_IHD-Ischaemic_heart_disease_wide_definition
(361194, 20857.0)


I9_MI-Myocardial_infarction
(361194, 7018.0)


I21-Diagnoses_main_ICD10_I21_Acute_myocardial_infarction
(361194, 5948.0)


20002_1075-Noncancer_illness_code_selfreported_heart_attackmyocardial_infarction
(361141, 8239.0)




## Get best tissue results for Niacin

In [33]:
drugs_tissue_df = {}

with pd.HDFStore(input_predictions_by_tissue_file, mode="r") as store:
    for tk in store.keys():
        df = store[tk][_drug_id]

        drugs_tissue_df[tk[1:]] = df

In [34]:
_tmp = pd.DataFrame(drugs_tissue_df)
display(_tmp.shape)
display(_tmp.head())

(4091, 49)

,Adipose_Subcutaneous,Adipose_Visceral_Omentum,Adrenal_Gland,Artery_Aorta,Artery_Coronary,Artery_Tibial,Brain_Amygdala,Brain_Anterior_cingulate_cortex_BA24,Brain_Caudate_basal_ganglia,Brain_Cerebellar_Hemisphere,...,Skin_Not_Sun_Exposed_Suprapubic,Skin_Sun_Exposed_Lower_leg,Small_Intestine_Terminal_Ileum,Spleen,Stomach,Testis,Thyroid,Uterus,Vagina,Whole_Blood
I9_PHLETHROMBDVTLOW-DVT_of_lower_extremities,2225108,3633306,3960107,1332054,2182424,4240339,2487093,2274668,1708839,2311499,...,3111573,1680886,2903454,1799322,1457560,3024874,3919867,3272210,1370451,3027555
I71-Diagnoses_main_ICD10_I71_Aortic_aneurysm_and_dissection,1803572,711574,1448934,4231713,1607177,1551651,1389767,3638570,2445540,3317543,...,1935886,2817997,1179611,1986576,1191470,2691960,1236895,3169182,1650408,2093344
G62-Diagnoses_main_ICD10_G62_Other_polyneuropathies,1298787,2518740,3957170,2937233,2458553,3229771,3576149,1369567,3312298,2019896,...,2011435,1944383,4573539,4539100,2696412,2205179,785903,1301301,4430058,3196323
2395_4-Hairbalding_pattern_Pattern_4,626108,3038228,2098431,2826578,308521,3097367,649715,2836304,3183412,871365,...,2939378,3161076,3586681,2400041,863019,928483,2868830,1913291,2029917,3557621
20003_1141168590-Treatmentmedication_code_pariet_10mg_ec_tablet,3017562,1880510,1094152,4206760,4452501,4179176,2875126,2042720,2486881,3534752,...,2561731,516964,2166641,2783551,2651308,2219414,1217317,2210372,3053042,3362472


In [35]:
# show top tissue models (from TWAS) for each trait
traits_best_tissues_df = (
    pd.DataFrame(drugs_tissue_df).loc[_phenomexcan_traits].idxmax(1)
)
display(traits_best_tissues_df)

I70-Diagnoses_main_ICD10_I70_Atherosclerosis                                                   Cells_Cultured_fibroblasts
CARDIoGRAM_C4D_CAD_ADDITIVE                                                                              Colon_Transverse
I25-Diagnoses_main_ICD10_I25_Chronic_ischaemic_heart_disease                                  Brain_Putamen_basal_ganglia
20002_1473-Noncancer_illness_code_selfreported_high_cholesterol                                            Brain_Amygdala
6150_100-Vascularheart_problems_diagnosed_by_doctor_None_of_the_above                                               Liver
6150_1-Vascularheart_problems_diagnosed_by_doctor_Heart_attack                                   Brain_Frontal_Cortex_BA9
I9_CHD-Major_coronary_heart_disease_event                                                     Brain_Cerebellar_Hemisphere
I9_CORATHER-Coronary_atherosclerosis                                                               Heart_Atrial_Appendage
I9_IHD-Ischaemic_heart_d

In [36]:
# pick the tissue with the maximum score for each trait
drug_df = pd.DataFrame(drugs_tissue_df).max(1)

In [37]:
drug_df.shape

(4091,)

In [38]:
drug_df.head()

I9_PHLETHROMBDVTLOW-DVT_of_lower_extremities                       4240339
I71-Diagnoses_main_ICD10_I71_Aortic_aneurysm_and_dissection        4231713
G62-Diagnoses_main_ICD10_G62_Other_polyneuropathies                4573539
2395_4-Hairbalding_pattern_Pattern_4                               3979892
20003_1141168590-Treatmentmedication_code_pariet_10mg_ec_tablet    4452501
dtype: int64

In [39]:
drug_df.loc[_phenomexcan_traits].sort_values()

20002_1075-Noncancer_illness_code_selfreported_heart_attackmyocardial_infarction    3996589
6150_1-Vascularheart_problems_diagnosed_by_doctor_Heart_attack                      4014872
I21-Diagnoses_main_ICD10_I21_Acute_myocardial_infarction                            4242830
I9_MI-Myocardial_infarction                                                         4417032
I9_CHD-Major_coronary_heart_disease_event                                           4443732
6150_100-Vascularheart_problems_diagnosed_by_doctor_None_of_the_above               4539449
20002_1473-Noncancer_illness_code_selfreported_high_cholesterol                     4549772
I70-Diagnoses_main_ICD10_I70_Atherosclerosis                                        4574511
CARDIoGRAM_C4D_CAD_ADDITIVE                                                         4598049
I9_IHD-Ischaemic_heart_disease_wide_definition                                      4671838
I25-Diagnoses_main_ICD10_I25_Chronic_ischaemic_heart_disease                    

In [40]:
drug_df.describe()

count    4.091000e+03
mean     4.240773e+06
std      3.346693e+05
min      2.317380e+06
25%      4.064316e+06
50%      4.323137e+06
75%      4.488742e+06
max      4.760801e+06
dtype: float64

In [41]:
drug_mean, drug_std = drug_df.mean(), drug_df.std()
display((drug_mean, drug_std))

(4240772.880958201, 334669.30609300843)

In [42]:
drug_df_stats = ((drug_df - drug_mean) / drug_std).describe()
display(drug_df_stats)

count    4.091000e+03
mean    -1.299159e-15
std      1.000000e+00
min     -5.747145e+00
25%     -5.272574e-01
50%      2.461060e-01
75%      7.409392e-01
max      1.553857e+00
dtype: float64

In [43]:
drug_df = (drug_df.loc[_phenomexcan_traits] - drug_mean) / drug_std

In [44]:
drug_df.shape

(12,)

In [45]:
drug_df.sort_values()

20002_1075-Noncancer_illness_code_selfreported_heart_attackmyocardial_infarction   -0.729627
6150_1-Vascularheart_problems_diagnosed_by_doctor_Heart_attack                     -0.674997
I21-Diagnoses_main_ICD10_I21_Acute_myocardial_infarction                            0.006147
I9_MI-Myocardial_infarction                                                         0.526667
I9_CHD-Major_coronary_heart_disease_event                                           0.606447
6150_100-Vascularheart_problems_diagnosed_by_doctor_None_of_the_above               0.892451
20002_1473-Noncancer_illness_code_selfreported_high_cholesterol                     0.923297
I70-Diagnoses_main_ICD10_I70_Atherosclerosis                                        0.997218
CARDIoGRAM_C4D_CAD_ADDITIVE                                                         1.067550
I9_IHD-Ischaemic_heart_disease_wide_definition                                      1.288033
I25-Diagnoses_main_ICD10_I25_Chronic_ischaemic_heart_disease          

All predictions of Niacin for these traits are high (above the mean and a standard deviation away)

In [46]:
# select traits for which niacin has a high prediction
selected_traits = drug_df[drug_df > drug_df_stats["75%"]].index.tolist()

In [47]:
selected_traits

['I70-Diagnoses_main_ICD10_I70_Atherosclerosis',
 'CARDIoGRAM_C4D_CAD_ADDITIVE',
 'I25-Diagnoses_main_ICD10_I25_Chronic_ischaemic_heart_disease',
 '20002_1473-Noncancer_illness_code_selfreported_high_cholesterol',
 '6150_100-Vascularheart_problems_diagnosed_by_doctor_None_of_the_above',
 'I9_CORATHER-Coronary_atherosclerosis',
 'I9_IHD-Ischaemic_heart_disease_wide_definition']

## Gene module-based - LVs driving association

In [48]:
def find_best_tissue(trait_id):
    return traits_best_tissues_df.loc[trait_id]

In [49]:
_tmp_res = find_best_tissue("I9_CORATHER-Coronary_atherosclerosis")
display(_tmp_res)

'Heart_Atrial_Appendage'

In [50]:
# available_doids = set(predictions_by_tissue["trait"].unique())
traits_lv_data = []

for trait in selected_traits:
    best_module_tissue = find_best_tissue(trait)
    display(best_module_tissue)

    best_module_tissue_data = pd.read_pickle(
        conf.RESULTS["DRUG_DISEASE_ANALYSES"]
        / "spredixcan"
        / "proj"
        / f"spredixcan-mashr-zscores-{best_module_tissue}-projection.pkl"
    )[trait]

    traits_lv_data.append(best_module_tissue_data)

'Cells_Cultured_fibroblasts'

'Colon_Transverse'

'Brain_Putamen_basal_ganglia'

'Brain_Amygdala'

'Liver'

'Heart_Atrial_Appendage'

'Brain_Cerebellar_Hemisphere'

In [51]:
module_tissue_data = pd.DataFrame(traits_lv_data).T

In [52]:
module_tissue_data.shape

(987, 7)

In [53]:
module_tissue_data.head()

,I70-Diagnoses_main_ICD10_I70_Atherosclerosis,CARDIoGRAM_C4D_CAD_ADDITIVE,I25-Diagnoses_main_ICD10_I25_Chronic_ischaemic_heart_disease,20002_1473-Noncancer_illness_code_selfreported_high_cholesterol,6150_100-Vascularheart_problems_diagnosed_by_doctor_None_of_the_above,I9_CORATHER-Coronary_atherosclerosis,I9_IHD-Ischaemic_heart_disease_wide_definition
LV1,0.005720,-0.002536,0.003223,0.027183,-0.009975,-0.041645,-0.034174
LV2,-0.002927,-0.040910,0.009556,0.031170,-0.000601,-0.053181,-0.025907
LV3,0.002734,0.033692,-0.005872,-0.015628,0.029873,-0.018676,-0.020514
LV4,0.015454,-0.017444,0.004483,0.000231,-0.073387,0.022468,-0.046343
LV5,0.027796,0.016727,-0.009114,-0.037452,0.032017,-0.030727,-0.015489


In [54]:
drug_data = lincs_projection.loc[_drug_id]

In [70]:
drug_data.head()

LV1   -0.032338
LV2    0.007960
LV3    0.009136
LV4   -0.005717
LV5   -0.023626
Name: DB00627, dtype: float64

In [56]:
_tmp = (-1.0 * drug_data.dot(module_tissue_data)).sort_values(ascending=False)
display(_tmp)

I9_IHD-Ischaemic_heart_disease_wide_definition                           0.041041
6150_100-Vascularheart_problems_diagnosed_by_doctor_None_of_the_above    0.033455
I9_CORATHER-Coronary_atherosclerosis                                     0.025396
I70-Diagnoses_main_ICD10_I70_Atherosclerosis                             0.023000
I25-Diagnoses_main_ICD10_I25_Chronic_ischaemic_heart_disease             0.021002
20002_1473-Noncancer_illness_code_selfreported_high_cholesterol          0.018520
CARDIoGRAM_C4D_CAD_ADDITIVE                                              0.016894
Name: DB00627, dtype: float64

In [57]:
drug_trait_predictions = pd.DataFrame(
    -1.0 * (drug_data.to_frame().values * module_tissue_data.values),
    columns=module_tissue_data.columns.copy(),
    index=drug_data.index.copy(),
)

In [58]:
drug_trait_predictions.shape

(987, 7)

In [59]:
drug_trait_predictions.head()

,I70-Diagnoses_main_ICD10_I70_Atherosclerosis,CARDIoGRAM_C4D_CAD_ADDITIVE,I25-Diagnoses_main_ICD10_I25_Chronic_ischaemic_heart_disease,20002_1473-Noncancer_illness_code_selfreported_high_cholesterol,6150_100-Vascularheart_problems_diagnosed_by_doctor_None_of_the_above,I9_CORATHER-Coronary_atherosclerosis,I9_IHD-Ischaemic_heart_disease_wide_definition
LV1,0.000185,-0.000082,0.000104,0.000879,-0.000323,-0.001347,-0.001105
LV2,0.000023,0.000326,-0.000076,-0.000248,0.000005,0.000423,0.000206
LV3,-0.000025,-0.000308,0.000054,0.000143,-0.000273,0.000171,0.000187
LV4,0.000088,-0.000100,0.000026,0.000001,-0.000420,0.000128,-0.000265
LV5,0.000657,0.000395,-0.000215,-0.000885,0.000756,-0.000726,-0.000366


In [60]:
common_lvs = []

for c in drug_trait_predictions.columns:
    _tmp = drug_trait_predictions[c]

    _tmp = _tmp[_tmp > 0.0]
    q = _tmp.quantile(QUANTILE)
    _tmp = _tmp[_tmp > q]
    display(f"Number of LVs: {_tmp.shape[0]}")

    _tmp = _tmp.sort_values(ascending=False)
    common_lvs.append(_tmp)

    display(_tmp.head(20))
    print()

'Number of LVs: 25'

LV116    0.006180
LV931    0.004843
LV744    0.002292
LV697    0.002231
LV885    0.001984
LV536    0.001956
LV550    0.001933
LV220    0.001822
LV272    0.001785
LV739    0.001770
LV678    0.001525
LV470    0.001444
LV66     0.001375
LV189    0.001371
LV517    0.001296
LV840    0.001285
LV246    0.001253
LV502    0.001249
LV525    0.001227
LV85     0.001220
Name: I70-Diagnoses_main_ICD10_I70_Atherosclerosis, dtype: float64

'Number of LVs: 25'

LV536    0.005978
LV74     0.004517
LV879    0.003454
LV38     0.002886
LV841    0.002169
LV530    0.001887
LV163    0.001600
LV678    0.001565
LV824    0.001456
LV58     0.001449
LV256    0.001403
LV59     0.001383
LV95     0.001338
LV104    0.001336
LV136    0.001332
LV97     0.001327
LV189    0.001298
LV239    0.001296
LV170    0.001292
LV885    0.001170
Name: CARDIoGRAM_C4D_CAD_ADDITIVE, dtype: float64

'Number of LVs: 25'

...

LV743    0.006904
LV969    0.004285
LV575    0.004086
LV829    0.003023
LV707    0.002743
LV840    0.002685
LV931    0.002528
LV254    0.002311
LV885    0.002250
LV656    0.002098
LV216    0.001912
LV95     0.001887
LV366    0.001873
LV739    0.001867
LV290    0.001829
LV227    0.001828
LV125    0.001815
LV461    0.001811
LV841    0.001794
LV408    0.001696
Name: 6150_100-Vascularheart_problems_diagnosed_by_doctor_None_of_the_above, dtype: float64

'Number of LVs: 25'

LV881    0.015670
LV670    0.002761
LV824    0.002730
LV142    0.002600
LV227    0.002379
LV707    0.002191
LV530    0.002034
LV841    0.001971
LV254    0.001843
LV97     0.001810
LV21     0.001728
LV163    0.001628
LV974    0.001596
LV313    0.001535
LV282    0.001472
LV674    0.001425
LV678    0.001410
LV468    0.001377
LV346    0.001357
LV354    0.001353
Name: I9_CORATHER-Coronary_atherosclerosis, dtype: float64

'Number of LVs: 26'

LV881    0.007724
LV97     0.004228
LV931    0.003626
LV509    0.003360
LV395    0.002838
LV479    0.001992
LV502    0.001909
LV536    0.001816
LV220    0.001572
LV351    0.001518
LV570    0.001508
LV601    0.001426
LV116    0.001423
LV170    0.001326
LV906    0.001322
LV246    0.001268
LV78     0.001254
LV354    0.001240
LV310    0.001206
LV824    0.001194
Name: I9_IHD-Ischaemic_heart_disease_wide_definition, dtype: float64

# Niacin top LVs

In [71]:
drug_data.abs().sort_values(ascending=False).head(30)

LV881    0.154953
LV116    0.097558
LV743    0.097299
LV38     0.088232
LV841    0.086833
LV163    0.080580
LV161    0.070249
LV931    0.069832
LV9      0.069249
LV536    0.068436
LV50     0.067657
LV707    0.067467
LV142    0.064834
LV95     0.061820
LV97     0.061123
LV74     0.056454
LV66     0.055342
LV974    0.053342
LV678    0.051296
LV879    0.050419
LV969    0.047919
LV27     0.047240
LV509    0.046626
LV272    0.046522
LV905    0.045610
LV254    0.044915
LV720    0.043503
LV587    0.043327
LV608    0.043223
LV246    0.042009
Name: DB00627, dtype: float64

In [72]:
drug_data.sort_values(ascending=False).head(15)

LV881    0.154953
LV116    0.097558
LV743    0.097299
LV38     0.088232
LV974    0.053342
LV678    0.051296
LV879    0.050419
LV272    0.046522
LV254    0.044915
LV720    0.043503
LV587    0.043327
LV20     0.040827
LV840    0.039595
LV507    0.037171
LV684    0.034499
Name: DB00627, dtype: float64

In [73]:
drug_data.sort_values(ascending=True).head(15)

LV841   -0.086833
LV163   -0.080580
LV161   -0.070249
LV931   -0.069832
LV9     -0.069249
LV536   -0.068436
LV50    -0.067657
LV707   -0.067467
LV142   -0.064834
LV95    -0.061820
LV97    -0.061123
LV74    -0.056454
LV66    -0.055342
LV969   -0.047919
LV27    -0.047240
Name: DB00627, dtype: float64

# Get common LVs

In [61]:
common_lvs_df = (
    pd.concat(common_lvs).reset_index().rename(columns={"index": "lv", 0: "value"})
)

In [62]:
common_lvs_df.shape

(177, 2)

In [63]:
common_lvs_df.head()

,lv,value
0,LV116,0.006180
1,LV931,0.004843
2,LV744,0.002292
3,LV697,0.002231
4,LV885,0.001984


In [68]:
lvs_by_sum = common_lvs_df.groupby("lv").sum().squeeze().sort_values(ascending=False)
display(lvs_by_sum.head(25))

lv
LV881    0.036456
LV116    0.014301
LV97     0.011069
LV931    0.010996
LV163    0.010162
LV536    0.009750
LV227    0.009540
LV707    0.007616
LV743    0.006904
LV885    0.006806
LV824    0.006586
LV530    0.006100
LV678    0.005976
LV841    0.005933
LV74     0.005811
LV254    0.005603
LV95     0.005427
LV840    0.005103
LV170    0.004578
LV395    0.004369
LV246    0.004369
LV969    0.004285
LV575    0.004086
LV38     0.003989
LV21     0.003789
Name: value, dtype: float64

In [69]:
lvs_by_count = (
    common_lvs_df.groupby("lv").count().squeeze().sort_values(ascending=False)
)
display(lvs_by_count.head(25))

lv
LV97     5
LV707    4
LV227    4
LV116    4
LV678    4
LV824    4
LV163    4
LV885    4
LV246    3
LV21     3
LV840    3
LV841    3
LV354    3
LV254    3
LV881    3
LV536    3
LV530    3
LV931    3
LV95     3
LV170    3
LV395    2
LV38     2
LV525    2
LV587    2
LV59     2
Name: value, dtype: int64

# Save

In [66]:
output_file = OUTPUT_DIR / "cardiovascular-niacin.h5"
display(output_file)

PosixPath('/media/miltondp/Elements1/projects/phenoplier/results/drug_disease_analyses/lincs/analyses/cardiovascular-niacin.h5')

In [67]:
with pd.HDFStore(output_file, mode="w", complevel=4) as store:
    store.put("traits_module_tissue_data", module_tissue_data, format="fixed")
    store.put("drug_data", drug_data, format="fixed")
    store.put("drug_trait_predictions", drug_trait_predictions, format="fixed")
    store.put("common_lvs", common_lvs_df, format="fixed")